<a href="https://colab.research.google.com/github/eduseiti/ia368v_dd_final/blob/master/create_clueweb22_pt_colbertx_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys

In [2]:
IN_COLAB='google.colab' in sys.modules

In [3]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)

    !pip install transformers -q
    !pip install mlflow -q
    !pip install ujson -q
    !pip install faiss-gpu -q
    !pip install SentencePiece -q

    if not os.path.exists("ColBERT-X"):
        !git clone https://github.com/hltcoe/ColBERT-X.git

    PYTHON="python3"
else:
    WORKING_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/trabalho_final/"

    PỲTHON="python3.8"

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62

In [4]:
os.chdir(WORKING_FOLDER)

In [5]:
import json
import glob
import tqdm
import re
import time
import shutil

### Define the corpus part index to create the index

In [6]:
COLBERT_CORPUS_PART_INDEX=0

### Define other constants

In [7]:
COLBERTX_CORPUS_ORIGINAL_FOLDER="clueweb22-pt_10M_sample"
COLBERTX_CORPUS_VM_FOLDER="/content"

COLBERTX_INDEX_FOLDER="clueweb22-pt_10M_sample_part_{:02}_index".format(COLBERT_CORPUS_PART_INDEX)

COLBERT_CORPUS_PART_FILENAME="clueweb22-pt_colbertx_{:02}.tsv".format(COLBERT_CORPUS_PART_INDEX)

### Copy the corpus part file to the local VM to speedup the index creation process

In [11]:
start_time = time.time()

shutil.copy(os.path.join(COLBERTX_CORPUS_ORIGINAL_FOLDER, COLBERT_CORPUS_PART_FILENAME), COLBERTX_CORPUS_VM_FOLDER)

print("Time to copy the corpus file: {}".format(time.time() - start_time))

Time to copy the corpus file: 73.84907341003418


### Create the colbertx index

In [12]:
os.chdir("ColBERT-X")

In [ ]:
start_time = time.time()

!python3 -m xlmr_colbert.index --similarity l2 \
		--checkpoint ../colbertx_experiments/mMSMARCO-pt_048_dim/train.py/fine_tune_025/checkpoints/colbert-20000.dnn \
		--index_root ../{COLBERTX_CORPUS_ORIGINAL_FOLDER} --index_name {COLBERTX_INDEX_FOLDER} \
		--collection {COLBERTX_CORPUS_VM_FOLDER}/{COLBERT_CORPUS_PART_FILENAME} --doc_maxlen 480 --query_maxlen 32 --dim 48 --amp --bsize=512 --chunksize=24

2023-06-18 18:09:44.418248: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-18 18:09:44.471767: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-18 18:09:45.441057: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[Jun 18, 18:10:03] #> Creating directory /content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR/ColBERT-X/experiments/dirty/index.py/2023-06-18_18.09.47 




[Jun 18, 18:10:13] #> Creating directory /content/drive/MyDrive/unicamp/ia368v

In [ ]:
print("Total time to create the ColBERT-X index: {}".format(time.time() - start_time))

### Now, generate the FAISS index

In [ ]:
start_time = time.time()

In [ ]:
!python3 -m xlmr_colbert.index_faiss --index_root ../{COLBERTX_CORPUS_ORIGINAL_FOLDER} --index_name {COLBERTX_INDEX_FOLDER}

In [ ]:
print("Total time to create the FAISS index: {}".format(time.time() - start_time))